- **`resume_data` collection** (from `resume_ale.md`): work experience, education, skills, continuing studies with header based chunking
- **`personality` collection** (from `personalities_16.md`): personality traits with fixed-size chunking
- **`projects` collection** (from `portfolio_projects.md`): portfolio projects with hierarchical chunking

## Initialize Vector Store Connection

In [1]:
from qdrant_client import QdrantClient
from qdrant_client.models import Filter, FieldCondition, MatchValue
from pathlib import Path
import json

# Initialize OpenAI embeddings for semantic search
import sys
sys.path.append('..')
from src.core.embeddings import OpenAIEmbeddings

embedder = OpenAIEmbeddings()
print("✅ Embedder initialized for semantic search queries")

# Initialize Qdrant client with local storage
storage_path = "../vector_db/qdrant_storage"
client = QdrantClient(path=storage_path)

# Collection names (triple-collection architecture)
resume_collection = "resume_data"
personality_collection = "personality"
projects_collection = "projects"

print("✅ Connected to Qdrant vector store")
print(f"📂 Storage path: {Path(storage_path).absolute()}")
print(f"\n📦 Collections:")
print(f"   - {resume_collection} (resume content)")
print(f"   - {personality_collection} (personality traits)")
print(f"   - {projects_collection} (portfolio projects)")

✅ Embedder initialized for semantic search queries
✅ Connected to Qdrant vector store
📂 Storage path: c:\Users\Ale\Documents\Data-Projects\GitHub\Resume_Claude_SDK_Agent\notebooks\..\vector_db\qdrant_storage

📦 Collections:
   - resume_data (resume content)
   - personality (personality traits)
   - projects (portfolio projects)


## Explore Collections Overview

In [2]:
# Get all collections
collections = client.get_collections()
print("📚 Available Collections:")
for collection in collections.collections:
    print(f"   - {collection.name}")

print("\n" + "="*80)

# Explore ALL collections
for collection_name in [resume_collection, personality_collection, projects_collection]:
    if client.collection_exists(collection_name):
        collection_info = client.get_collection(collection_name)
        
        print(f"\n📊 Collection '{collection_name}' Details:")
        print(f"   Total documents: {collection_info.points_count}")
        print(f"   Vector dimensions: {collection_info.config.params.vectors.size}")
        print(f"   Distance metric: {collection_info.config.params.vectors.distance}")
        print(f"   Status: {collection_info.status}")
        
        # Count by section type
        from collections import Counter
        all_records, _ = client.scroll(
            collection_name=collection_name,
            limit=1000,
            with_payload=True,
            with_vectors=False
        )
        
        section_counts = Counter(r.payload.get('section_type', 'unknown') for r in all_records)
        
        print(f"\n   📈 Documents by Section Type:")
        for section, count in sorted(section_counts.items()):
            print(f"      {section:20s}: {count:3d} chunks")
        
        print("   " + "-"*76)
    else:
        print(f"\n❌ Collection '{collection_name}' not found")

print("\n" + "="*80)

📚 Available Collections:
   - resume_data
   - personality
   - projects


📊 Collection 'resume_data' Details:
   Total documents: 35
   Vector dimensions: 1536
   Distance metric: Cosine
   Status: green

   📈 Documents by Section Type:
      continuing_studies  :   7 chunks
      education           :   2 chunks
      personal_info       :   1 chunks
      professional_summary:   1 chunks
      skills              :   5 chunks
      work_experience     :  19 chunks
   ----------------------------------------------------------------------------

📊 Collection 'personality' Details:
   Total documents: 8
   Vector dimensions: 1536
   Distance metric: Cosine
   Status: green

   📈 Documents by Section Type:
                          :   8 chunks
   ----------------------------------------------------------------------------

📊 Collection 'projects' Details:
   Total documents: 42
   Vector dimensions: 1536
   Distance metric: Cosine
   Status: green

   📈 Documents by Section Type:
     

## Resume Data Queries

### Work Experience with Full Metadata

In [3]:
# Filter for work experience entries (from resume_data collection)
work_filter = Filter(
    must=[
        FieldCondition(
            key="section_type",
            match=MatchValue(value="work_experience")
        )
    ]
)

work_records, _ = client.scroll(
    collection_name=resume_collection,  # Query resume_data collection
    scroll_filter=work_filter,
    limit=20,
    with_payload=True,
    with_vectors=False  # Set True to see embeddings
)

print(f"💼 Work Experience Chunks from '{resume_collection}' collection (showing {len(work_records)}):\n")

for i, record in enumerate(work_records, 1):
    payload = record.payload
    metadata = payload.get('metadata', {})
    
    print(f"{'='*80}")
    print(f"CHUNK {i} - ID: {record.id}")
    print(f"{'='*80}")
    print(f"📄 Content (Achievement):")
    print(f"   {payload.get('content', 'N/A')}")
    print(f"\n🏢 Metadata:")
    print(f"   Company:        {metadata.get('company', 'N/A')}")
    print(f"   Position:       {metadata.get('position', 'N/A')}")
    print(f"   Start Date:     {metadata.get('start_date', 'N/A')}")
    print(f"   End Date:       {metadata.get('end_date', 'N/A')}")
    print(f"   Source File:    {payload.get('source_file', 'N/A')}")
    print(f"   Section Type:   {payload.get('section_type', 'N/A')}")
    print()

💼 Work Experience Chunks from 'resume_data' collection (showing 19):

CHUNK 1 - ID: 0b2b312a-3c2d-4a6d-a6af-cad1bc2a98bc
📄 Content (Achievement):
   Quality Assurance Technician: Coordinated supply chain and production teams to ensure food safety compliance by leading cross-functional meetings and implementing compliance checks, maintaining operational continuity.

🏢 Metadata:
   Company:        The Very Good Food Company
   Position:       Quality Assurance Technician
   Start Date:     February-2021
   End Date:       February-2022
   Source File:    resume_ale.md
   Section Type:   work_experience

CHUNK 2 - ID: 120e8829-f4e5-4f0e-aba9-00c0632d6772
📄 Content (Achievement):
   Data Scientist II: Developed a Power BI dashboard to track changes in imported food volumes, collaborating with import inspectors to define metrics and design visualizations in Power BI for stakeholder use.

🏢 Metadata:
   Company:        Canadian Food Inspection Agency
   Position:       Data Scientist II
   S

### Work Experience with Embeddings

Each chunk has a 1536-dimensional embedding vector generated by OpenAI's `text-embedding-3-small` model.

In [4]:
# Get one work experience record WITH embeddings
work_with_vector, _ = client.scroll(
    collection_name=resume_collection,
    scroll_filter=work_filter,
    limit=20,
    with_payload=True,
    with_vectors=True  # Include embeddings
)

if work_with_vector:
    record = work_with_vector[0]
    vector = record.vector
    
    print(f"🔢 Embedding Vector Details:")
    print(f"   Vector dimensions: {len(vector)}")
    print(f"   Vector type: {type(vector)}")
    print(f"   First 10 values: {vector[:10]}")
    print(f"   Last 10 values:  {vector[-10:]}")
    print(f"\n📊 Vector Statistics:")
    import numpy as np
    vector_array = np.array(vector)
    print(f"   Min value:  {vector_array.min():.6f}")
    print(f"   Max value:  {vector_array.max():.6f}")
    print(f"   Mean value: {vector_array.mean():.6f}")
    print(f"   Std dev:    {vector_array.std():.6f}")
    
    print(f"\n📄 Associated Content:")
    print(f"   {record.payload.get('content', 'N/A')[:150]}...")

🔢 Embedding Vector Details:
   Vector dimensions: 1536
   Vector type: <class 'list'>
   First 10 values: [-0.019191697239875793, 0.011832953430712223, 0.06691659241914749, 0.00033292826265096664, -0.016301382333040237, 0.029689325019717216, 0.020486559718847275, 0.019688831642270088, 0.03362015634775162, 0.01056121475994587]
   Last 10 values:  [0.024139918386936188, 0.00474011804908514, -0.013596045784652233, -0.0012016488471999764, -0.01553833857178688, -0.006566797848790884, -0.006665068678557873, 0.014879346825182438, 0.003659140085801482, -0.0037920945324003696]

📊 Vector Statistics:
   Min value:  -0.097485
   Max value:  0.109878
   Mean value: 0.000860
   Std dev:    0.025501

📄 Associated Content:
   Quality Assurance Technician: Coordinated supply chain and production teams to ensure food safety compliance by leading cross-functional meetings and ...


### Education and Skills

Query education entries and skills from the resume data collection.

In [5]:
# Query education entries (from resume_data collection)
education_filter = Filter(
    must=[FieldCondition(key="section_type", match=MatchValue(value="education"))]
)

education_records, _ = client.scroll(
    collection_name=resume_collection,  # ← Query resume_data collection
    scroll_filter=education_filter,
    limit=20,
    with_payload=True
)

print(f"🎓 Education Entries from '{resume_collection}' collection ({len(education_records)}):\n")
for i, record in enumerate(education_records, 1):
    payload = record.payload
    metadata = payload.get('metadata', {})
    
    print(f"{'='*80}")
    print(f"EDUCATION CHUNK {i}")
    print(f"{'='*80}")
    print(f"📝 Degree:        {metadata.get('degree', 'N/A')}")
    print(f"🏫 Institution:   {metadata.get('institution', 'N/A')}")
    print(f"📅 Year:          {metadata.get('year', 'N/A')}")
    print(f"📂 Source File:   {payload.get('source_file', 'N/A')}")
    print(f"🏷️  Section Type:  {payload.get('section_type', 'N/A')}")
    print(f"\n📄 Content:\n   {payload.get('content', 'N/A')}")
    print(f"\n🔍 Full Metadata: {json.dumps(metadata, indent=2)}")
    print()

# Query skills (from resume_data collection)
skills_filter = Filter(
    must=[FieldCondition(key="section_type", match=MatchValue(value="skills"))]
)

skills_records, _ = client.scroll(
    collection_name=resume_collection,
    scroll_filter=skills_filter,
    limit=20,
    with_payload=True
)

print(f"\n🛠️  Skills Entries from '{resume_collection}' collection ({len(skills_records)}):\n")
for i, record in enumerate(skills_records, 1):
    payload = record.payload
    metadata = payload.get('metadata', {})
    
    print(f"{'='*80}")
    print(f"SKILL CHUNK {i}")
    print(f"{'='*80}")
    print(f"📂 Category:      {metadata.get('category', 'N/A')}")
    print(f"📄 Skills:        {payload.get('content', 'N/A')}")
    print(f"📁 Source File:   {payload.get('source_file', 'N/A')}")
    print(f"🏷️  Section Type:  {payload.get('section_type', 'N/A')}")
    print(f"\n🔍 Full Metadata: {json.dumps(metadata, indent=2)}")
    print()

🎓 Education Entries from 'resume_data' collection (2):

EDUCATION CHUNK 1
📝 Degree:        BSc in Biotechnology Engineering
🏫 Institution:   Tec de Monterrey
📅 Year:          N/A
📂 Source File:   resume_ale.md
🏷️  Section Type:  education

📄 Content:
   BSc in Biotechnology Engineering from Tec de Monterrey. August-2012 - May-2017 | Mexico

🔍 Full Metadata: {
  "degree": "BSc in Biotechnology Engineering",
  "institution": "Tec de Monterrey",
  "dates": "August-2012 - May-2017 | Mexico"
}

EDUCATION CHUNK 2
📝 Degree:        MSc in Food Science
🏫 Institution:   University of British Columbia
📅 Year:          N/A
📂 Source File:   resume_ale.md
🏷️  Section Type:  education

📄 Content:
   MSc in Food Science from University of British Columbia. January-2019 - October-2020 | Canada

🔍 Full Metadata: {
  "degree": "MSc in Food Science",
  "institution": "University of British Columbia",
  "dates": "January-2019 - October-2020 | Canada"
}


🛠️  Skills Entries from 'resume_data' collection (5)

## Personality Traits Queries

### Personality Sections

In [6]:
personality_filter = Filter(
    must=[FieldCondition(key="section_type", match=MatchValue(value=""))]
)

personality_records, _ = client.scroll(
    collection_name=personality_collection,
    scroll_filter=personality_filter,
    limit=20,
    with_payload=True
)

print(f"🧠 Personality Trait Chunks from '{personality_collection}' collection ({len(personality_records)}):\n")
print(f"💡 Note: This collection contains ONLY personality data with simplified fixed-size chunking\n")

for i, record in enumerate(personality_records, 1):
    payload = record.payload
    metadata = payload.get('metadata', {})
    
    print(f"{'='*80}")
    print(f"PERSONALITY CHUNK {i}")
    print(f"{'='*80}")
    print(f"📝 Chunk Index: {metadata.get('chunk_index', 'N/A')}")
    print(f"📂 Source File: {payload.get('source_file', 'N/A')}")
    print(f"📏 Character Range: {metadata.get('char_start', 'N/A')} - {metadata.get('char_end', 'N/A')}")
    print(f"\n📄 Content:\n   {payload.get('content', 'N/A')}")
    print(f"\n🔍 Full Metadata: {json.dumps(metadata, indent=2)}")
    print()

🧠 Personality Trait Chunks from 'personality' collection (8):

💡 Note: This collection contains ONLY personality data with simplified fixed-size chunking

PERSONALITY CHUNK 1
📝 Chunk Index: 3
📂 Source File: personalities_16.md
📏 Character Range: 900 - 1300

📄 Content:
   also attending to crucial details, which makes me a valuable asset in any organization.

## Strengths
### Innovative Mindset
My ability to see possibilities others overlook often helps me find smarter solutions and effective improvements at work.

### Independent Worker
My talent for working productively on my own allows me to manage tasks effectively without the need for constant direction or sup

🔍 Full Metadata: {
  "chunk_index": 3,
  "char_start": 900,
  "char_end": 1300,
  "overlap_chars": 100
}

PERSONALITY CHUNK 2
📝 Chunk Index: 0
📂 Source File: personalities_16.md
📏 Character Range: 0 - 400

📄 Content:
   # Personality Traits
I'm very analytical, highly curious and constantly seek to improve systems. I approac

In [7]:
# Complete workflow simulation
print(f"{'='*80}")
print("COMPLETE PROJECTS RETRIEVAL WORKFLOW")
print(f"{'='*80}\n")

job_tech_requirements = "data visualization matplotlib seaborn statistical analysis"

print(f"📋 Job Requirements: {job_tech_requirements}\n")

# PHASE 1: Search technical summaries
print(f"{'='*80}")
print("PHASE 1: Search Technical Summaries")
print(f"{'='*80}\n")

query_vector = embedder.embed_query(job_tech_requirements)

tech_results = client.query_points(
    collection_name=projects_collection,
    query=query_vector,
    query_filter=Filter(
        must=[FieldCondition(key="section_type", match=MatchValue(value="project_technical"))]
    ),
    limit=3,
    score_threshold=0.3  # Minimum similarity
).points

print(f"✅ Found {len(tech_results)} matching projects:\n")

matched_project_ids = []
for i, result in enumerate(tech_results, 1):
    metadata = result.payload.get('metadata', {})
    project_id = metadata.get('project_id')
    matched_project_ids.append(project_id)
    
    print(f"{i}. [{result.score:.3f}] {metadata.get('project_title', 'N/A')}")
    print(f"   ID: {project_id}")
    print(f"   Tech: {', '.join(metadata.get('tech_stack', []))}\n")

# PHASE 2: Retrieve full content for matched projects
print(f"\n{'='*80}")
print("PHASE 2: Retrieve Full Project Content")
print(f"{'='*80}\n")

print(f"🔍 Retrieving full content for {len(matched_project_ids)} projects...\n")

for project_id in matched_project_ids:
    # Filter for full content of this project
    full_filter = Filter(
        must=[
            FieldCondition(key="metadata.project_id", match=MatchValue(value=project_id)),
            FieldCondition(key="section_type", match=MatchValue(value="project_full"))
        ]
    )
    
    full_results, _ = client.scroll(
        collection_name=projects_collection,
        scroll_filter=full_filter,
        limit=1,
        with_payload=True,
        with_vectors=False
    )
    
    if full_results:
        record = full_results[0]
        payload = record.payload
        metadata = payload.get('metadata', {})
        
        print(f"{'='*80}")
        print(f"PROJECT: {metadata.get('project_title', 'N/A')}")
        print(f"{'='*80}")
        print(f"🆔 ID: {project_id}")
        print(f"💻 Tech Stack: {', '.join(metadata.get('tech_stack', []))}")
        print(f"🔗 URL: {metadata.get('project_url', 'N/A')}")
        print(f"\n📄 Full Content (first 600 chars):")
        print(payload.get('content', 'N/A')[:600])
        print("...\n")

print(f"\n{'='*80}")
print("✅ WORKFLOW COMPLETE")
print(f"{'='*80}")
print("\n💡 Benefits of Hierarchical Chunking:")
print("   ✓ Fast initial matching using technical summaries")
print("   ✓ Retrieve full context only for relevant projects")
print("   ✓ Efficient token usage (don't embed full content for initial search)")
print("   ✓ Clear separation of filtering vs. detailed context")

COMPLETE PROJECTS RETRIEVAL WORKFLOW

📋 Job Requirements: data visualization matplotlib seaborn statistical analysis

PHASE 1: Search Technical Summaries

✅ Found 3 matching projects:

1. [0.526] CERN Career Preferences Analysis: A Data Visualization
   ID: project_3
   Tech: Python, Pandas, Seaborn, Matplotlib

2. [0.509] AI Model Performance Visualization: Grouped Bar vs. Bubble Charts
   ID: project_2
   Tech: Python, Pandas, Plotly, Matplotlib

3. [0.492] Canadian Grain Production Analysis: Visualizing Data with Optimal Color Scales
   ID: project_1
   Tech: Python, Pandas, Plotly, Matplotlib, NumPy


PHASE 2: Retrieve Full Project Content

🔍 Retrieving full content for 3 projects...

PROJECT: CERN Career Preferences Analysis: A Data Visualization
🆔 ID: project_3
💻 Tech Stack: Python, Pandas, Seaborn, Matplotlib
🔗 URL: https://tinyurl.com/hnexp2r8

📄 Full Content (first 600 chars):
# CERN Career Preferences Analysis: A Data Visualization

## Purpose
This project transforms observat

## Projects Collection Queries

### Complete Projects Retrieval Workflow

Demonstrates the two-phase retrieval:
1. **Phase 1**: Search `project_technical` for matching projects
2. **Phase 2**: Retrieve `project_full` content using project_ids

In [8]:
# Simulate job requirements
job_requirements = "Python data visualization pandas matplotlib plotly"

print(f"🔍 Semantic Search: Finding projects matching job requirements")
print(f"{'='*80}\n")
print(f"Query: '{job_requirements}'")
print(f"Collection: {projects_collection}")
print(f"Target: project_technical (for fast matching)\n")

# Generate query embedding
query_vector = embedder.embed_query(job_requirements)

# Search technical summaries first (faster, focused)
tech_filter = Filter(
    must=[FieldCondition(key="section_type", match=MatchValue(value="project_technical"))]
)

tech_results = client.query_points(
    collection_name=projects_collection,
    query=query_vector,
    query_filter=tech_filter,
    limit=5
).points

print(f"✅ Top {len(tech_results)} Matching Projects (by technical summary):\n")

for i, result in enumerate(tech_results, 1):
    payload = result.payload
    metadata = payload.get('metadata', {})
    
    print(f"{'='*80}")
    print(f"MATCH {i} - Similarity Score: {result.score:.4f}")
    print(f"{'='*80}")
    print(f"📦 Project: {metadata.get('project_title', 'N/A')}")
    print(f"🆔 ID: {metadata.get('project_id', 'N/A')}")
    print(f"💻 Tech Stack: {', '.join(metadata.get('tech_stack', []))}")
    print(f"🔗 URL: {metadata.get('project_url', 'N/A')}")
    print(f"\n📄 Technical Summary (first 300 chars):")
    print(f"   {payload.get('content', 'N/A')[:300]}...")
    print()

print("\n💡 Workflow: Search technical summaries → Get project_id → Retrieve full content using metadata filter")

🔍 Semantic Search: Finding projects matching job requirements

Query: 'Python data visualization pandas matplotlib plotly'
Collection: projects
Target: project_technical (for fast matching)

✅ Top 5 Matching Projects (by technical summary):

MATCH 1 - Similarity Score: 0.5283
📦 Project: CERN Career Preferences Analysis: A Data Visualization
🆔 ID: project_3
💻 Tech Stack: Python, Pandas, Seaborn, Matplotlib
🔗 URL: https://tinyurl.com/hnexp2r8

📄 Technical Summary (first 300 chars):
   Project: CERN Career Preferences Analysis: A Data Visualization

Technologies: Python, Pandas, Seaborn, Matplotlib

Technical Work:
The project demonstrates proficiency in data transformation and visualization with pandas for dataset organization and sorting. Custom styling techniques are applied th...

MATCH 2 - Similarity Score: 0.5170
📦 Project: AI Model Performance Visualization: Grouped Bar vs. Bubble Charts
🆔 ID: project_2
💻 Tech Stack: Python, Pandas, Plotly, Matplotlib
🔗 URL: https://tinyurl.com/4xk

### Semantic Search: Find Projects by Technical Requirements

Search for projects matching specific technologies or technical work.

In [9]:
# Query for a specific project using metadata filtering
target_project_id = "project_0"  # Change this to test different projects

print(f"🔍 Searching for project by metadata: project_id = '{target_project_id}'")
print(f"{'='*80}\n")

# Build filter for exact metadata match
metadata_filter = Filter(
    must=[
        FieldCondition(
            key="metadata.project_id",
            match=MatchValue(value=target_project_id)
        )
    ]
)

# Retrieve matching documents
results, _ = client.scroll(
    collection_name=projects_collection,
    scroll_filter=metadata_filter,
    limit=10,
    with_payload=True,
    with_vectors=False
)

print(f"✅ Found {len(results)} chunks for {target_project_id}\n")

for i, record in enumerate(results, 1):
    payload = record.payload
    metadata = payload.get('metadata', {})
    section_type = payload.get('section_type', 'N/A')
    
    print(f"{'='*80}")
    print(f"CHUNK {i}: {section_type}")
    print(f"{'='*80}")
    print(f"📦 Project Title: {metadata.get('project_title', 'N/A')}")
    print(f"🆔 Project ID: {metadata.get('project_id', 'N/A')}")
    print(f"💻 Tech Stack: {', '.join(metadata.get('tech_stack', []))}")
    print(f"📦 Chunk Type: {metadata.get('chunk_type', 'N/A')}")
    
    if section_type == "project_technical":
        print(f"\n🔧 Technical Summary (first 400 chars):")
        print(f"   {payload.get('content', 'N/A')[:400]}...")
    else:
        print(f"\n📚 Full Content (first 500 chars):")
        print(f"   {payload.get('content', 'N/A')[:500]}...")
    print()

print("\n💡 Use Case: Retrieve full project details after finding relevant technical summary")

🔍 Searching for project by metadata: project_id = 'project_0'

✅ Found 2 chunks for project_0

CHUNK 1: project_technical
📦 Project Title: Wage Gap BC Minimum vs. Living Wage 2025
🆔 Project ID: project_0
💻 Tech Stack: Python, Pandas, NumPy, Seaborn, Matplotlib, Jupyter Notebook
📦 Chunk Type: technical_summary

🔧 Technical Summary (first 400 chars):
   Project: Wage Gap BC Minimum vs. Living Wage 2025

Technologies: Python, Pandas, NumPy, Seaborn, Matplotlib, Jupyter Notebook

Technical Work:
Implements a slope chart visualization technique—a sophisticated data storytelling approach that effectively communicates the magnitude of wage disparity between two data points. The project demonstrates advanced matplotlib customization including custom ax...

CHUNK 2: project_full
📦 Project Title: Wage Gap BC Minimum vs. Living Wage 2025
🆔 Project ID: project_0
💻 Tech Stack: Python, Pandas, NumPy, Seaborn, Matplotlib, Jupyter Notebook
📦 Chunk Type: full_content

📚 Full Content (first 500 chars):


### Query by Metadata: Retrieve Specific Project by ID

In [10]:
# Filter for full content chunks
full_filter = Filter(
    must=[FieldCondition(key="section_type", match=MatchValue(value="project_full"))]
)

full_records, _ = client.scroll(
    collection_name=projects_collection,
    scroll_filter=full_filter,
    limit=1000,
    with_payload=True,
    with_vectors=False
)

print(f"📚 Full Project Content Chunks (project_full)")
print(f"{'='*80}\n")
print(f"💡 These chunks contain ALL sections for complete project context\n")

for i, record in enumerate(full_records, 1):
    payload = record.payload
    metadata = payload.get('metadata', {})
    print(f"{'='*80}")
    print(f"PROJECT {i}: {metadata.get('project_title', 'N/A')}")
    print(f"{'='*80}")
    print(f"🆔 Project ID: {metadata.get('project_id', 'N/A')}")
    print(f"🔗 URL: {metadata.get('project_url', 'N/A')}")
    print(f"💻 Tech Stack: {', '.join(metadata.get('tech_stack', []))}")
    print(f"📦 Chunk Type: {metadata.get('chunk_type', 'N/A')}")
    print(f"🏷️  Section Type: {payload.get('section_type', 'N/A')}")
    print(f"\n📄 Full Content")
    print(payload.get('content', 'N/A'))
    print(f"\n{'='*80}\n")

# Summary of all full projects
print(f"\n📊 Summary of All Full Project Chunks ({len(full_records)} total):\n")
for i, record in enumerate(full_records, 1):
    metadata = record.payload.get('metadata', {})
    print(f"{i}. {metadata.get('project_title', 'N/A')}")
    print(f"   Tech: {', '.join(metadata.get('tech_stack', []))}")
    print()

📚 Full Project Content Chunks (project_full)

💡 These chunks contain ALL sections for complete project context

PROJECT 1: Customer Review Sentiment Analysis
🆔 Project ID: project_8
🔗 URL: https://tinyurl.com/3baruh6x
💻 Tech Stack: Python, Streamlit, Transformers, PyTorch, NLTK, Pandas, NumPy, Matplotlib, Seaborn, Scipy
📦 Chunk Type: full_content
🏷️  Section Type: project_full

📄 Full Content
# Customer Review Sentiment Analysis

## Purpose
This project decodes emotional opinions in customer reviews using advanced natural language processing techniques, enabling businesses to refine marketing strategies, optimize product offerings, and identify emerging market trends. By automatically classifying sentiment across large volumes of customer feedback, it provides actionable intelligence for competitive positioning in consumer goods industries.

## Tech Stack
Python, Streamlit, Transformers, PyTorch, NLTK, Pandas, NumPy, Matplotlib, Seaborn, Scipy

## Technical Highlights
The project imple

### View Full Project Content (project_full chunks)

In [11]:
# Filter for technical summary chunks
tech_filter = Filter(
    must=[FieldCondition(key="section_type", match=MatchValue(value="project_technical"))]
)

tech_records, _ = client.scroll(
    collection_name=projects_collection,
    scroll_filter=tech_filter,
    limit=100,
    with_payload=True,
    with_vectors=False
)

print(f"🔧 Technical Summary Chunks (project_technical)")
print(f"{'='*80}\n")
print(f"💡 These chunks contain Tech Stack + Technical Highlights for fast matching\n")

for i, record in enumerate(tech_records, 1):
    payload = record.payload
    metadata = payload.get('metadata', {})
    
    print(f"{'='*80}")
    print(f"PROJECT {i}: {metadata.get('project_title', 'N/A')}")
    print(f"{'='*80}")
    print(f"🆔 Project ID: {metadata.get('project_id', 'N/A')}")
    print(f"🔗 URL: {metadata.get('project_url', 'N/A')}")
    print(f"💻 Tech Stack: {', '.join(metadata.get('tech_stack', []))}")
    print(f"📦 Chunk Type: {metadata.get('chunk_type', 'N/A')}")
    print(f"🏷️  Section Type: {payload.get('section_type', 'N/A')}")
    print(f"\n📄 Content Preview (first 300 chars):")
    print(f"   {payload.get('content', 'N/A')[:300]}...")
    print()

🔧 Technical Summary Chunks (project_technical)

💡 These chunks contain Tech Stack + Technical Highlights for fast matching

PROJECT 1: Optimizing Ad Campaign: 62% Reduction in Ad Spend
🆔 Project ID: project_7
🔗 URL: https://tinyurl.com/ms8vv7my
💻 Tech Stack: Python, Pandas, NumPy, Seaborn, Jupyter Notebook, Power BI
📦 Chunk Type: technical_summary
🏷️  Section Type: project_technical

📄 Content Preview (first 300 chars):
   Project: Optimizing Ad Campaign: 62% Reduction in Ad Spend

Technologies: Python, Pandas, NumPy, Seaborn, Jupyter Notebook, Power BI

Technical Work:
The analysis implements comprehensive data preprocessing, cleaning missing values and standardizing data formats from raw campaign data containing mul...

PROJECT 2: Air Quality Index ETL to ML Prediction Pipeline
🆔 Project ID: project_5
🔗 URL: https://tinyurl.com/3xdwhxes
💻 Tech Stack: Python, Ploomber, DuckDB, MotherDuck, OpenAQ API, ARIMA, Pandas, Streamlit, Statsmodels, Jupyter Notebooks, Docker, GitHub Actions
📦 Ch

### View Technical Summaries (project_technical chunks)

In [12]:
# View all projects in the collection
all_projects, _ = client.scroll(
    collection_name=projects_collection,
    limit=100,
    with_payload=True,
    with_vectors=False
)

print(f"📂 Projects Collection Overview")
print(f"{'='*80}\n")
print(f"Total chunks in collection: {len(all_projects)}")

# Group by project_id
from collections import defaultdict
projects_by_id = defaultdict(list)
for record in all_projects:
    project_id = record.payload.get('metadata', {}).get('project_id')
    projects_by_id[project_id].append(record)

print(f"Total unique projects: {len(projects_by_id)}")
print(f"\n💡 Each project has 2 chunks: technical_summary + full_content")
print(f"\n{'='*80}\n")

# Display each project's chunks
for project_id, chunks in sorted(projects_by_id.items()):
    tech_chunk = next((c for c in chunks if c.payload.get('section_type') == 'project_technical'), None)
    full_chunk = next((c for c in chunks if c.payload.get('section_type') == 'project_full'), None)
    
    if tech_chunk:
        metadata = tech_chunk.payload.get('metadata', {})
        print(f"📦 Project: {metadata.get('project_title', 'N/A')}")
        print(f"   ID: {project_id}")
        print(f"   URL: {metadata.get('project_url', 'N/A')}")
        print(f"   Tech Stack: {', '.join(metadata.get('tech_stack', []))}")
        print(f"   Chunks: {len(chunks)} (technical + full)")
        print()

📂 Projects Collection Overview

Total chunks in collection: 42
Total unique projects: 21

💡 Each project has 2 chunks: technical_summary + full_content


📦 Project: Wage Gap BC Minimum vs. Living Wage 2025
   ID: project_0
   URL: https://tinyurl.com/yjy6vdfr
   Tech Stack: Python, Pandas, NumPy, Seaborn, Matplotlib, Jupyter Notebook
   Chunks: 2 (technical + full)

📦 Project: Canadian Grain Production Analysis: Visualizing Data with Optimal Color Scales
   ID: project_1
   URL: https://tinyurl.com/2wxtb6af
   Tech Stack: Python, Pandas, Plotly, Matplotlib, NumPy
   Chunks: 2 (technical + full)

📦 Project: Google Diet Search Trends 2020 - Canada Geographic Analytics Dashboard
   ID: project_10
   URL: https://tinyurl.com/mrxvv8un
   Tech Stack: Tableau
   Chunks: 2 (technical + full)

📦 Project: Breakfast Cereals Nutritional Analysis Dashboard
   ID: project_11
   URL: https://tinyurl.com/52b3kr8t
   Tech Stack: Tableau
   Chunks: 2 (technical + full)

📦 Project: Supermarket Sales Dash

### Hierarchical Chunking Strategy

The `projects` collection uses a **hierarchical chunking strategy** with two chunk types per project:

1. **`project_technical`**: Tech Stack + Technical Highlights (for filtering/matching)
2. **`project_full`**: Complete project with all sections (Purpose, Tech Stack, Technical Highlights, Skills Demonstrated, Result/Impact)

In [13]:
# Create a query for Python-related achievements
query_text = "Python data analysis ETL pipeline machine learning"
query_vector = embedder.embed_query(query_text)

print(f"🔍 Semantic Search Query: '{query_text}'")
print(f"   Query vector dimensions: {len(query_vector)}")
print(f"   Searching in: {resume_collection} collection")

# Search with vector similarity using query_points (newer API)
results = client.query_points(
    collection_name=resume_collection,  # ← Query resume_data collection
    query=query_vector,
    limit=5,
    score_threshold=0.5  # Only return results with similarity > 0.5
).points

print(f"\n📊 Top {len(results)} Results (by semantic similarity):\n")

for i, result in enumerate(results, 1):
    payload = result.payload
    metadata = payload.get('metadata', {})
    
    print(f"{'='*80}")
    print(f"RESULT {i} - Similarity Score: {result.score:.4f}")
    print(f"{'='*80}")
    print(f"📄 Content: {payload.get('content', 'N/A')}")
    print(f"🏷️  Section Type: {payload.get('section_type', 'N/A')}")
    if payload.get('section_type') == 'work_experience':
        print(f"   Company: {metadata.get('company', 'N/A')}")
        print(f"   Position: {metadata.get('position', 'N/A')}")
    print()

🔍 Semantic Search Query: 'Python data analysis ETL pipeline machine learning'
   Query vector dimensions: 1536
   Searching in: resume_data collection

📊 Top 4 Results (by semantic similarity):

RESULT 1 - Similarity Score: 0.6346
📄 Content: Data Analyst: Built an ETL pipeline integrating five data sources totaling over 1M records using SQL and Python, automating ingestion and cleaning and saving 8 hours weekly in data preparation.
🏷️  Section Type: work_experience
   Company: Rubicon Organics
   Position: Data Analyst

RESULT 2 - Similarity Score: 0.5505
📄 Content: Data Scientist II: Extracted and processed millions of import/export transactions by building web-scraping collectors and a PySpark ETL pipeline to load cleaned data into a Microsoft Fabric lakehouse.
🏷️  Section Type: work_experience
   Company: Canadian Food Inspection Agency
   Position: Data Scientist II

RESULT 3 - Similarity Score: 0.5154
📄 Content: Data Scientist II: Automated data categorization, reducing data clean

### Search for Personality Traits Matching Job Requirements

In [14]:
# Simulate a job analysis with soft skills and keywords
job_analysis = {
    'soft_skills': ['analytical thinking', 'problem-solving', 'collaboration'],
    'keywords': ['strategic', 'innovative', 'team player']
}

# Build query (same logic as retrieve_personality_traits)
query_parts = job_analysis.get('soft_skills', []) + job_analysis.get('keywords', [])
query_text = ' '.join(query_parts)
query_vector = embedder.embed_query(query_text)

print(f"🔍 Job Requirements Query: '{query_text}'")
print(f"   Searching in: {personality_collection} collection (NEW!)\n")

# Search the PERSONALITY collection directly (no filtering needed!)
all_results = client.query_points(
    collection_name=personality_collection,  # ← Query personality collection directly!
    query=query_vector,
    limit=10
).points

print(f"✅ Retrieved {len(all_results)} results from personality collection")

print(f"\n📊 Top 5 Personality Trait Chunks by Semantic Similarity:\n")
print(f"💡 Benefits of simplified chunking:")
print(f"   - Pure semantic search without complex filtering")
print(f"   - Fixed-size chunks maintain consistent context windows")
print(f"   - Faster search (smaller collection)\n")

for i, result in enumerate(all_results[:5], 1):  # Top 5
    payload = result.payload
    metadata = payload.get('metadata', {})
    
    print(f"{'='*80}")
    print(f"TRAIT {i} - Similarity: {result.score:.4f}")
    print(f"{'='*80}")
    print(f"📝 Chunk Index: {metadata.get('chunk_index', 'N/A')}")
    print(f"📏 Characters: {metadata.get('char_start', 'N/A')} - {metadata.get('char_end', 'N/A')}")
    print(f"📄 Content:\n   {payload.get('content', 'N/A')}")
    print()

print("\n💡 These traits would be deduplicated (removing 100-char overlaps) and injected into the cover letter prompt!")

🔍 Job Requirements Query: 'analytical thinking problem-solving collaboration strategic innovative team player'
   Searching in: personality collection (NEW!)

✅ Retrieved 8 results from personality collection

📊 Top 5 Personality Trait Chunks by Semantic Similarity:

💡 Benefits of simplified chunking:
   - Pure semantic search without complex filtering
   - Fixed-size chunks maintain consistent context windows
   - Faster search (smaller collection)

TRAIT 1 - Similarity: 0.5586
📝 Chunk Index: 0
📏 Characters: 0 - 400
📄 Content:
   # Personality Traits
I'm very analytical, highly curious and constantly seek to improve systems. I approach life with a strategic mindset, always looking several steps ahead and planning for contingencies. I value autonomy but also enjoy collaborating with others. This allows me to tackle complex problems with confidence and innovation. I hold high standards for myself and others, always striving

TRAIT 2 - Similarity: 0.4968
📝 Chunk Index: 3
📏 Characters: 90

### Semantic Search with Section Filtering

In [15]:
# Search for data science achievements ONLY in work experience (resume_data collection)
query_text = "data science machine learning SQL Python dashboard visualization"
query_vector = embedder.embed_query(query_text)

# Apply filter to only search work_experience
work_filter = Filter(
    must=[FieldCondition(key="section_type", match=MatchValue(value="work_experience"))]
)

results = client.query_points(
    collection_name=resume_collection,  # ← Query resume_data collection
    query=query_vector,
    query_filter=work_filter,  # ← Apply filter during search
    limit=5
).points

print(f"🔍 Query: '{query_text}'")
print(f"📦 Collection: {resume_collection}")
print(f"🎯 Filter: section_type = 'work_experience'")
print(f"\n📊 Top {len(results)} Work Achievements:\n")

for i, result in enumerate(results, 1):
    payload = result.payload
    metadata = payload.get('metadata', {})
    
    print(f"{i}. [Score: {result.score:.4f}] {metadata.get('company', 'N/A')} - {metadata.get('position', 'N/A')}")
    print(f"   {payload.get('content', 'N/A')[:100]}...")
    print()

🔍 Query: 'data science machine learning SQL Python dashboard visualization'
📦 Collection: resume_data
🎯 Filter: section_type = 'work_experience'

📊 Top 5 Work Achievements:

1. [Score: 0.5813] Canadian Food Inspection Agency - Data Scientist
   Data Scientist: Standardized descriptive and statistical reporting in Power BI, reducing report-gene...

2. [Score: 0.5107] Rubicon Organics - Data Analyst
   Data Analyst: Built an ETL pipeline integrating five data sources totaling over 1M records using SQL...

3. [Score: 0.5098] Rubicon Organics - Data Analyst
   Data Analyst: Built three Power BI dashboards for sales and marketing by collaborating with stakehol...

4. [Score: 0.4946] Canadian Food Inspection Agency - Data Scientist II
   Data Scientist II: Automated forecasting and reduced manual effort by 40 hours per month by deployin...

5. [Score: 0.4918] Canadian Food Inspection Agency - Data Scientist II
   Data Scientist II: Implemented daily automated data refreshes, replacing weekly